In [1]:
import tempfile
from repo2graph.core import add_node_by_repo

In [2]:
repo_url = 'https://github.com/gitpython-developers/GitPython.git'

g = add_node_by_repo(repo_url=repo_url)

In [4]:
g.nodes

NodeView(('GitPython', 'GitPython/test', 'GitPython/test/test_imports.py', 'GitPython/test/test_config.py', 'GitPython/test/test_git.py', 'GitPython/test/test_fun.py', 'GitPython/test/test_quick_doc.py', 'GitPython/test/tstrunner.py', 'GitPython/test/test_tree.py', 'GitPython/test/test_db.py', 'GitPython/test/test_stats.py', 'GitPython/test/__init__.py', 'GitPython/test/test_index.py', 'GitPython/test/test_actor.py', 'GitPython/test/test_installation.py', 'GitPython/test/test_base.py', 'GitPython/test/test_util.py', 'GitPython/test/test_exc.py', 'GitPython/test/test_submodule.py', 'GitPython/test/test_blob_filter.py', 'GitPython/test/test_reflog.py', 'GitPython/test/test_diff.py', 'GitPython/test/test_refs.py', 'GitPython/test/test_commit.py', 'GitPython/test/test_docs.py', 'GitPython/test/test_repo.py', 'GitPython/test/test_remote.py', 'GitPython/test/test_clone.py', 'GitPython/test/test_blob.py', 'GitPython/test/performance', 'GitPython/test/performance/test_odb.py', 'GitPython/test/

In [5]:
print(g.nodes['GitPython/git/objects/submodule/util.py']['source'])

# This module is part of GitPython and is released under the
# 3-Clause BSD License: https://opensource.org/license/bsd-3-clause/

__all__ = [
    "sm_section",
    "sm_name",
    "mkhead",
    "find_first_remote_branch",
    "SubmoduleConfigParser",
]

from io import BytesIO
import weakref

import git
from git.config import GitConfigParser
from git.exc import InvalidGitRepositoryError

# typing -----------------------------------------------------------------------

from typing import Any, Sequence, TYPE_CHECKING, Union

from git.types import PathLike

if TYPE_CHECKING:
    from weakref import ReferenceType

    from git.refs import Head, RemoteReference
    from git.remote import Remote
    from git.repo import Repo

    from .base import Submodule

# { Utilities


def sm_section(name: str) -> str:
    """:return: Section title used in ``.gitmodules`` configuration file"""
    return f'submodule "{name}"'


def sm_name(section: str) -> str:
    """:return: Name of the submodule as pa

In [ ]:
import tree_sitter_python as tspython
from tree_sitter import Language, Parser

PY_LANGUAGE = Language(tspython.language())

In [7]:
parser = Parser(PY_LANGUAGE)

In [8]:
tree = parser.parse(
    bytes(
        g.nodes['GitPython/git/objects/submodule/util.py']['source'],
        "utf8"
    )
)

In [11]:
root_node = tree.root_node

In [51]:
import networkx as nx
import networkx as nx

import networkx as nx

def extract_class_function_relationships_with_source(g, root_node):
    """
    Extract class and function relationships from a syntax tree into a given graph.

    Args:
        g (nx.DiGraph): A pre-existing directed graph.
        root_node: The root node of the syntax tree, which contains the `source` attribute.
    """
    def get_source_text(node):
        """Extract the source code corresponding to a node."""
        source = root_node.source  # Use source from the root_node
        return source[node.start_byte:node.end_byte]
    
    def traverse(node, current_parent):
        if node.type in ['class_definition', 'function_definition']:
            name_node = node.child_by_field_name('name')
            if name_node:
                name = root_node.source[name_node.start_byte:name_node.end_byte]
                full_name = '/'.join([current_parent, name]) if current_parent else name  # Construct hierarchical name
                source_text = get_source_text(node)
                node_type = 'class' if node.type == 'class_definition' else 'function'
                
                # Add node with attributes
                g.add_node(full_name, 
                           type=node_type, 
                           parent=current_parent, 
                           source=source_text, 
                           is_class=(node_type == 'class'))
                
                # Connect to the parent if present
                if current_parent:
                    g.add_edge(current_parent, full_name)

                # If it's a class, traverse its body to extract methods
                if node_type == 'class':
                    body_node = node.child_by_field_name('body')
                    if body_node:  # Recursively traverse the class body
                        traverse(body_node, full_name)

            # Set current_parent to full_name for child functions or classes
            current_parent = full_name if node.type == 'class_definition' else current_parent

        # Recursively traverse other children
        for child in node.children:
            traverse(child, current_parent)
    
    traverse(root_node, None)
    return g




In [47]:
graph = extract_class_function_relationships_with_source(tree, 'GitPython/git/objects/submodule/util.py', g.nodes['GitPython/git/objects/submodule/util.py']['source'])

In [48]:
graph.nodes

NodeView(('GitPython/git/objects/submodule/util.py/sm_section', 'GitPython/git/objects/submodule/util.py/sm_name', 'GitPython/git/objects/submodule/util.py/mkhead', 'GitPython/git/objects/submodule/util.py/find_first_remote_branch', 'GitPython/git/objects/submodule/util.py/SubmoduleConfigParser', 'GitPython/git/objects/submodule/util.py/SubmoduleConfigParser/__init__', 'GitPython/git/objects/submodule/util.py/SubmoduleConfigParser/set_submodule', 'GitPython/git/objects/submodule/util.py/SubmoduleConfigParser/flush_to_index', 'GitPython/git/objects/submodule/util.py/SubmoduleConfigParser/write'))

In [50]:
print(graph.nodes['GitPython/git/objects/submodule/util.py/SubmoduleConfigParser/__init__']['source'])

def __init__(self, *args: Any, **kwargs: Any) -> None:
        self._smref: Union["ReferenceType[Submodule]", None] = None
        self._index = None
        self._auto_write = True
        super().__init__(*args, **kwargs)
